In [1]:
import pandas as pd
import numpy as np

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import optuna

In [2]:
df = pd.read_csv('train.csv')
df_pred = pd.read_csv('test.csv', index_col='id')
sample = pd.read_csv('sample_submission.csv')

col = df.columns[1:]
df_more = pd.read_csv('Faults.NNA', header=None, sep='\t', names=col)

In [3]:
df = pd.concat([df, df_more], axis=0)

In [4]:
df.drop(columns='id', inplace=True)

In [5]:
lst_target = ['Pastry', 'Z_Scratch', 'K_Scatch',
              'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
X = df.drop(columns=lst_target)
y = np.argmax(df[lst_target], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# def objective(trial: optuna.Trial):
#     '''
#     Обычная сигнатура функции оптимизации для optuna.
#     '''
#     param = {
#             #"objective": "multiclass",
#             #"metric": "multi_logloss",
#             "verbosity": -1,
#             "boosting_type": "gbdt",
#             "random_state": 42,
#             "num_class": 7,
#             "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
#             "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
#             'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
#             'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
#             "max_depth": trial.suggest_int("max_depth", 5, 25),
#             "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
#             "subsample": trial.suggest_float("subsample", 0.8, 1.0),
#             "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
#             'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
#             'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0)}
    
#     lgbm_model = LGBMClassifier(**param)
#     lgbm_model.fit(X_train, y_train)
#     y_pred = lgbm_model.predict_proba(X_test)
#     return roc_auc_score(y_test, y_pred, multi_class="ovr")

In [7]:
# %%time
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)
# study.best_params

In [8]:
params ={
        #"objective": "multiclass",
        #"metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "num_class": 7,
        'learning_rate': 0.010796336532769835,
        'n_estimators': 364,
        'lambda_l1': 1.971431159270062e-08,
        'lambda_l2': 2.969222950399536e-08,
        'max_depth': 6,
        'colsample_bytree': 0.6549848799184741,
        'subsample': 0.8955542124621352,
        'min_child_samples': 11,
        'feature_fraction': 0.48325162720615855,
        'bagging_fraction': 0.9424422755677117
}

In [9]:
lgbm_model = LGBMClassifier(**params)
lgbm_model.fit(X_train, y_train)
y_pred = lgbm_model.predict_proba(X_test)
print(roc_auc_score(y_test, y_pred, multi_class="ovr"))

0.8825733651031602


In [10]:
lgbm_model.fit(X, y)
pred = lgbm_model.predict_proba(df_pred)

In [11]:
submission = pd.DataFrame(pred, columns=lst_target, index=df_pred.index)
submission.to_csv('model.csv', index=True)

# kaggle = 0.89149